## Testing the speed difference between ExLlamaV2 vs Orthodox Transformer

In [ ]:
!apt update

In [ ]:
!pip install --upgrade transformers safetensors sentencepiece huggingface-hub protobuf accelerate bitsandbytes tqdm openai backoff retrying ipykernel ipywidgets matplotlib cmake scikit-build-core setuptools

  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
Loading exllamav2_ext extension (JIT)...

Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Loading translation file:  manual_prompts_translated\ProntoQA\naive_prompting_explanations_before_answer.txt
SAMPLE PCT: 1
START INDEX: None type: <class 'NoneType'>
Loaded 5 examples from dev split.
Number of batch:  1
Running example id:  ProntoQA_1

Naive prompting: Deskripsi Tugas: Anda diberikan sebuah konteks yang berisi sekumpulan premis dan pertanyaan mengenai konteks tersebut. Tugas Anda adalah menjawab pertanyaan tersebut dengan memilih salah satu nilai kebenaran True atau False berdasarkan konteks yang diberikan dengan memberikan penjelasan langkah-langkah yang Anda gunakan untuk mencapai nilai kebenaran/jawaban tersebut.

------

Contoh:

Konteks:
Impuses adalah besar. Alex adalah impus.

Pertanyaan:
Alex 

g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\venv\lib\site-packages\setuptools\_distutils\_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(

  0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  25%|██▌       | 1/4 [00:03<00:10,  3.39s/it]

Loading checkpoint shards:  50%|█████     | 2/4 [00:05<00:05,  2.92s/it]

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:22<00:09,  9.27s/it]

Loading checkpoint shards: 100%|██████████| 4/4 [00:30<00:00,  7.67s/it]

 40%|████      | 2/5 [25:45<38:35, 771.99s/it]


In [ ]:
!pip install vllm

In [ ]:
!pip install llama-cpp-python \
    --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 \
    --force-reinstall

In [ ]:
!pip install "numpy<2.3"

In [ ]:
#!pip install --no-cache-dir --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu128

In [ ]:
!apt install git-lfs

In [ ]:
import os
import sys

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# ######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["GIT_LFS_SKIP_SMUDGE"] = "1"

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/Qwen/Qwen2.5-7B-Instruct LLM_MODELS/Qwen2.5-7B-Instruct

In [ ]:
!git clone https://huggingface.co/Qwen/Qwen2.5-7B-Instruct-GGUF LLM_MODELS_GGUF/Qwen2.5-7B-Instruct-GGUF

In [ ]:
!cd LLM_MODELS_GGUF/Qwen2.5-7B-Instruct-GGUF && git lfs pull --include "*q4_k_m*"

In [ ]:
# 1. Download the official pre-built binaries (Ubuntu x64)
print("Downloading llama.cpp CLI tools...")
!wget -q https://github.com/ggml-org/llama.cpp/releases/download/b7134/llama-b7134-bin-ubuntu-x64.zip -O llama_tools.zip

# 2. Unzip into a specific folder
print("Extracting...")
!unzip -o -q llama_tools.zip -d llama_tools_bin

# 3. Make the splitter executable
tool_path = os.path.abspath("llama_tools_bin/build/bin/llama-gguf-split")
!chmod +x {tool_path}

# 4. Add to PATH environment variable so you can use '!' commands
if "llama_tools_bin/build/bin" not in os.environ["PATH"]:
    os.environ["PATH"] += os.pathsep + os.path.dirname(tool_path)

print(f"Installed tools to: {os.path.dirname(tool_path)}")

# 5. Verify it works
print("\nVerifying llama-gguf-split version:")
!llama-gguf-split --help | head -n 5

In [ ]:
!llama-gguf-split --merge LLM_MODELS_GGUF/Qwen2.5-7B-Instruct-GGUF/qwen2.5-7b-instruct-q4_k_m-00001-of-00002.gguf LLM_MODELS_GGUF/Qwen2.5-7B-Instruct-GGUF/qwen2.5-7b-instruct-q4_k_m.gguf

In [ ]:
from misc.speed_test_backend import BenchmarkConfig, run_benchmark

config = BenchmarkConfig(
    model_path="LLM_MODELS/Qwen2.5-7B-Instruct",
    backend="hf",
    quantize_4bit=True,
)

results = run_benchmark(config)

config = BenchmarkConfig(
    model_path="LLM_MODELS/Qwen2.5-7B-Instruct",
    backend="vllm",
    quantization="fp8",
)

results = run_benchmark(config)

config = BenchmarkConfig(
    model_path="LLM_MODELS_GGUF/Qwen2.5-7B-Instruct-GGUF/qwen2.5-7b-instruct-q4_k_m.gguf",
    backend="llamacpp",
)

results = run_benchmark(config)

### On specific dataset and prompt

In [9]:
import os
import sys

In [10]:
# Commandline args universal
# MAX_NEW_TOKENS is purely for text generation count limit while max_position_embeddings is for context_length based on LLM config.json. !!! input_length + MAX_NEW_TOKENS shopuld be < context_length, otherwise LLM breaks. Llama 3 only has 8k context length/max_posiiton_embedding. SEALIONv3-LLama3-8B-IT uses ROPE, max_position_embeddings follows ROPE limit 131k, Qwen2.5-7B-IT has 32k context length, SahabatAIv1-LLama3-8B-IT has 8k context length.
# Counted the response for each steps in notebook output cell with tokens counter online, translations ~400 tokens, ~decomposition ~500 tokens, search_resolve ~700 tokens
# Change this every process (translate, decompose, search_resolve), different value is needed. Time is in seconds.
os.environ["LLM_WORKER_MAX_TIME"] = "300"
LLM_WORKER_MAX_TIME=300
os.environ["MAX_NEW_TOKENS"] = "1200"
MAX_NEW_TOKENS=1200
os.environ["BATCH_NUM"] = "1"
BATCH_NUM=1

print("LLM_WORKER_MAX_TIME =", os.environ["LLM_WORKER_MAX_TIME"])
print("MAX_NEW_TOKENS =", os.environ["MAX_NEW_TOKENS"])
print("BATCH_NUM =", os.environ["BATCH_NUM"])

LLM_WORKER_MAX_TIME = 300
MAX_NEW_TOKENS = 1200
BATCH_NUM = 1


In [11]:
os.environ["LLM_BACKEND"] = "hf"

snapshot_path = "LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct"
#snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

### If kernel doesnt recognize
LLM_MODEL=snapshot_path
LOCAL_MODEL_PATH=snapshot_path


LOCAL_MODEL_PATH = LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct
LLM_MODEL = LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct


In [12]:
# Naive prompting only requires True or False answer based on context
os.environ["MAX_NEW_TOKENS"] = "600"
MAX_NEW_TOKENS=600
os.environ["BATCH_NUM"] = "1"
BATCH_NUM=1

In [13]:
# Solving with naive prompting with explanations only
!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_explanations_before_answer --split dev --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

Loading exllamav2_ext extension (JIT)...

Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Loading translation file:  manual_prompts_translated\ProntoQA\naive_prompting_explanations_before_answer.txt
SAMPLE PCT: 0
START INDEX: None type: <class 'NoneType'>
Loaded 1 examples from dev split.
Number of batch:  1
Running example id:  ProntoQA_1

Naive prompting: Deskripsi Tugas: Anda diberikan sebuah konteks yang berisi sekumpulan premis dan pertanyaan mengenai konteks tersebut. Tugas Anda adalah menjawab pertanyaan tersebut dengan memilih salah satu nilai kebenaran True atau False berdasarkan konteks yang diberikan dengan memberikan penjelasan langkah-langkah yang Anda gunakan untuk mencapai nilai kebenaran/jawaban tersebut.

------

Contoh:

Konteks:
Impuses adalah besar. Alex adalah impus.

Pertanyaan:
Alex besar

###
**Penjelasan**:
Alex adalah impus.
Impuses adalah besar.
Ale

g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\venv\lib\site-packages\setuptools\_distutils\_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(

  0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  25%|██▌       | 1/4 [00:02<00:07,  2.54s/it]

Loading checkpoint shards:  50%|█████     | 2/4 [00:05<00:05,  2.56s/it]

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:07<00:02,  2.58s/it]

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.04s/it]

100%|██████████| 1/1 [12:33<00:00, 753.13s/it]


In [14]:
!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --evaluation_method naive_prompting

Loading exllamav2_ext extension (JIT)...

Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Evaluating using the naive prompting method.
Counter: 3
ID: ProntoQA_1, GT: B, Negated: B, Non-negated: B
ID: ProntoQA_2, GT: A, Negated: A, Non-negated: A
Total instances: 2
Accuracy: 100.00%
Error id:  []
Correct id:  ['ProntoQA_1', 'ProntoQA_2']
Saved table image to: results_translated_naive_prompting/prompt_explanations_before_answer\ProntoQA\LLM_MODELS_llama3-8b-cpt-sahabatai-v1-instruct_results_table.png


g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\venv\lib\site-packages\setuptools\_distutils\_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(


In [15]:
# # Setting to vllm backend
# os.environ["LLM_BACKEND"] = "vllm"

# snapshot_path = "LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct"
# #snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT"

# os.environ["LOCAL_MODEL_PATH"] = snapshot_path
# os.environ["LLM_MODEL"] = snapshot_path
# os.environ["QUANTIZATION"] = "fp8"

# print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
# print("LLM_MODEL =", os.environ["LLM_MODEL"])

# ### If kernel doesnt recognize
# LLM_MODEL=snapshot_path
# LOCAL_MODEL_PATH=snapshot_path

In [16]:
# # Solving with naive prompting with explanations with vllm backend
# !python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_explanations_before_answer --split dev --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [17]:
# !python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --evaluation_method naive_prompting

In [18]:
# Setting to llamacpp backend
os.environ["LLM_BACKEND"] = "llamacpp"

snapshot_path = "LLM_MODELS_GGUF/SahabatAI/llama3-8b-cpt-sahabatai-v1-instruct-q4_k_m.gguf"
#snapshot_path = "/workspace/LLM_MODELS_EXL2/Llama-SEA-LION-v3-8B-IT-EXL2-Indonesia-Focus"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quants (and bitsandbytes is set up)
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

### If kernel doesnt recognize
LLM_MODEL=snapshot_path
LOCAL_MODEL_PATH=snapshot_path

LOCAL_MODEL_PATH = LLM_MODELS_GGUF/SahabatAI/llama3-8b-cpt-sahabatai-v1-instruct-q4_k_m.gguf
LLM_MODEL = LLM_MODELS_GGUF/SahabatAI/llama3-8b-cpt-sahabatai-v1-instruct-q4_k_m.gguf


In [19]:
# Solving with naive prompting with explanations with llamacpp backend
!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_explanations_before_answer --split dev --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

Loading exllamav2_ext extension (JIT)...

Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Loading translation file:  manual_prompts_translated\ProntoQA\naive_prompting_explanations_before_answer.txt
SAMPLE PCT: 0
START INDEX: None type: <class 'NoneType'>
Loaded 1 examples from dev split.
Number of batch:  1
Running example id:  ProntoQA_1

Naive prompting: Deskripsi Tugas: Anda diberikan sebuah konteks yang berisi sekumpulan premis dan pertanyaan mengenai konteks tersebut. Tugas Anda adalah menjawab pertanyaan tersebut dengan memilih salah satu nilai kebenaran True atau False berdasarkan konteks yang diberikan dengan memberikan penjelasan langkah-langkah yang Anda gunakan untuk mencapai nilai kebenaran/jawaban tersebut.

------

Contoh:

Konteks:
Impuses adalah besar. Alex adalah impus.

Pertanyaan:
Alex besar

###
**Penjelasan**:
Alex adalah impus.
Impuses adalah besar.
Ale

g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\venv\lib\site-packages\setuptools\_distutils\_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(
llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


In [20]:
!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --evaluation_method naive_prompting

Loading exllamav2_ext extension (JIT)...

Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Building C++/CUDA extension ------------------------------   0% 0:00:00 -:--:--
Evaluating using the naive prompting method.
Counter: 1
ID: ProntoQA_1, GT: B, Negated: B, Non-negated: B
Total instances: 1
Accuracy: 100.00%
Error id:  []
Correct id:  ['ProntoQA_1']
Saved table image to: results_translated_naive_prompting/prompt_explanations_before_answer\ProntoQA\LLM_MODELS_GGUF_SahabatAI_llama3-8b-cpt-sahabatai-v1-instruct-q4_k_m.gguf_results_table.png


g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\venv\lib\site-packages\setuptools\_distutils\_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(
